In [1]:
import pymupdf
import os
import pymupdf4llm
import getpass
from pydantic import BaseModel
import pandas as pd
import csv
import io
from pandasai import Agent

In [2]:
dic_path = '/Users/ashwanth/Data/Wines/May/Indents'

In [3]:
def get_all_files(dic_path):
    return os.listdir(dic_path)

In [4]:
files = get_all_files(dic_path)

In [5]:
def extract_data_from_file(file):
    doc = pymupdf.open(file)
    all_text = ""
    for page in doc:
        text = page.get_text()
        all_text += text
    return all_text

In [6]:
api_key = getpass.getpass("Enter API KEY: ")

Enter API KEY:  ········


In [8]:
from openai import OpenAI

client = OpenAI(
    api_key = api_key,
)


In [9]:
class InvoiceDataExtraction(BaseModel):
    data: pd.DataFrame

    class Config:
        arbitrary_types_allowed = True


In [10]:
all_data = []

for file in files:
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from the invoices and you return BrandName, Qty, Price in a csv format."},
        {"role": "user", "content": extract_data_from_file(os.path.join(dic_path,file))}
    ],
    #response_format=InvoiceDataExtraction,
    )
    csv_content = completion.choices[0].message.content.strip("```csv\n").strip("```")

    # Parse the CSV content
    csv_reader = csv.DictReader(io.StringIO(csv_content))
    data = pd.DataFrame(csv_reader, columns=['BrandName', 'Qty', 'Price'])
    all_data.append(data)
    

    

In [11]:
df = pd.concat(all_data, ignore_index=True, axis=0)

In [12]:
df.head()

,BrandName,Qty,Price
0,KING FISHER STRONG P,10,13010
1,KNOCKOUT HIGH PUNCH,30,39030
2,BUDWEISER KING OF BE,10,18010
3,BUDWEISER MAGNUM PRE,20,38020
4,KING FISHER PREMIUM,55,66055


In [13]:
df.to_csv('Stock.csv')

### Use OpenAI

In [22]:
while True:
    # Take input from the user
    user_input = input("Provide Brand Name -> ")

    # Check if the user wants to quit
    if user_input.lower() in ["exit", "quit"]:
        print("Terminating the process!")
        break

    try:
        completion = client.chat.completions.create(
          model="gpt-4o-2024-08-06",
          messages=[
            {"role": "system", "content": "You are a helpful assistant. Help me aggregate the qty from the following data and provide the qty for the brand {}".format(user_input)}, # <-- This is the system message that provides context to the model
            {"role": "user", "content": df.to_string()} 
          ]
        )
        
        print("Assistant: " + completion.choices[0].message.content)
    except Exception as e:
        print(f"Error: {e}")

Provide Brand Name ->  Black & White


Assistant: From the provided data, it seems you are interested in aggregating the quantity (`Qty`) specifically for the brand "Black & White". Based on the information, we look for entries in the "BrandName" column that include "Black & White" and sum the associated quantities. Here's how the aggregation is determined:

1. Locate entries for "Black White Blended" and "Black White Blende".
2. Sum the quantities for these entries.

Entries:
- `BLACK WHITE BLENDED` has a Qty of 1.
- `BLACK WHITE BLENDE` has a Qty of 1.

Calculating the total quantity for "Black & White":

Total Qty = 1 (BLACK WHITE BLENDED) + 1 (BLACK WHITE BLENDE) = 2

Therefore, the total quantity for the brand "Black & White" is 2.


Provide Brand Name ->  100 Piper


Assistant: The brand "Seagram's 100 Pipers" appears twice in the dataset. To find the total quantity (Qty) for "100 Pipers," we need to sum the Qty for both entries:

1. At index 89: Qty = 1
2. At index 100: Qty = 3

Total Qty for "Seagram's 100 Pipers" = 1 + 3 = **4**


Provide Brand Name ->  BSP


...

Assistant: To find the total quantity (Qty) for the brand "KF Strong," we need to consider all variations of its name as might be present in the dataset. In the given data, the brand "KF Strong" can appear as "KING FISHER STRONG" or any variations with similar patterns.

Let's go through the dataset entries for "KING FISHER STRONG" and its variations:
1. **KING FISHER STRONG P**: 
   - Row 0: Qty 10
   - Row 11: Qty 10
   - Row 37: Qty 10
   - Row 60: Qty 9
   - Row 91: Qty 10
   - Row 103: Qty 9

2. **KING FISHER STRONG**:
   - Row 116: Qty 10

3. **KING FISHER STRONG P 650ml** or longer variations:
   - Row 29 (including additional prefix/suffix as "(5017BSG)"): Qty 2
   - Row 143 (including additional prefix/suffix as "(5017BSG)"): Qty 7
   - Row 196 (including additional prefix/suffix as "(5017BSG) 650ml"): Qty 8

To calculate the total quantity for "KF Strong," sum up these quantities:

- From "KING FISHER STRONG P": \(10 + 10 + 10 + 9 + 10 + 9 = 58\)
- From "KING FISHER STRONG" v

Provide Brand Name ->  Hay 5000 Strong


Assistant: To find the aggregate quantity for the brand "Hay 5000 Strong", let's check your provided list against the correct brand name. It appears you may be referring to "HAYWARDS 5000 SUPER", which is the closest match based on your provided data.

Now, let's sum the quantities (Qty) for "HAYWARDS 5000 SUPER":

- At index 76: 7
- At index 122: 10
- At index 171: 10
- At index 183: 20
- At index 193: 5

Adding these together:

7 + 10 + 10 + 20 + 5 = 52

Therefore, the total quantity for "HAYWARDS 5000 SUPER" is 52.


Provide Brand Name ->  exit


Terminating the process!
